In [28]:
import typing
import math
import random

"""
XOR using neat
"""

class Genome:
    class Node:
        types = ['Input', 'Hidden', 'Output', 'Sensor']
        def __init__(self, type:tuple[str],num:int ,  active=True):
            self.inType= type[0]
            self.outType= type[1]
            self.num = num
        
        def __str__(self):
            return f"Node {self.inType} to {self.outType}. Number: {self.num}"

    class Connection:
        def __init__(self,input:int, out:int, weight:float,enabled:bool, innov:int):
            self.enabled= enabled
            self.input = input
            self.output = out
            self.weight = weight
            self.innov = innov

        def split_in_two(self, newNode:int, innov:int):
            in_conn = Genome.Connection(input=self.input, out=newNode, weight=1, enabled=True, innov= innov+1)
            out_conn = Genome.Connection(input=newNode, out=self.output, weight=self.weight, enabled=True, innov= innov+2)
            self.enabled = False
            return [in_conn, out_conn]

        def __str__(self):
            return f"Connection from {self.input} to {self.output}. Status: {self.enabled}"

    def __init__(self):
        self.node_genes : dict[int, Genome.Node] = {}
        self.connection_genes : list[Genome.Connection] = []

    def mutate_add_node(self):
        random_connection = self.connection_genes[random.randint(0, len(self.connection_genes)-1)]
        new_node_init = (
            self.node_genes[random_connection.input].outType,
            self.node_genes[random_connection.output].inType
            )
        node_innov = max(self.node_genes.keys()) + 1
        self.node_genes[node_innov] = Genome.Node(new_node_init, num=node_innov)
        conn_innov = len(self.connection_genes) - 1
        new_conns = random_connection.split_in_two(newNode=node_innov, innov=conn_innov )
        self.connection_genes += new_conns

    def mutate_add_connection(self):
        choices_in =  list(filter(None,[i if i.inType != 'Output' else None for i in  self.node_genes.values()]))
        choices_out = list(filter(None,[i if i.inType != 'Input' else None for i in  self.node_genes.values()]))
        choice_in = random.choice(choices_in)
        choice_out = random.choice(filter(lambda i : i != choice_in, choices_out))
        innov = len(self.connection_genes) -1
        new_conn = Genome.Connection(input=choice_in, out=choice_out, weight=random.random(), enabled=True, innov=innov)
        self.connection_genes.append(new_conn)

    def connect_nodes(self, idx1, idx2):
        new_conn = Genome.Connection(input=idx1, out=idx2, weight=1, enabled=True, innov=len(self.connection_genes))
        self.connection_genes.append(new_conn)
    
    def __str__(self):
        return f"Genome with {len(self.connection_genes)} connections and {len(self.node_genes.keys())} nodes\n {list(map(lambda x:x.__str__(),self.node_genes.values()))} \n {list(map(lambda x:x.__str__(), self.connection_genes))}]"

    def add_node(self, node):
        i = self.get_current_node_num() + 1 
        self.node_genes[i] = node
        return i
    
    def get_current_node_num(self) -> int:
        return max(self.node_genes.keys() or [0])


    @staticmethod
    def init_with_shape(shape:tuple[int]):
        g = Genome()
        input_size, output_size = shape
        inputs = []
        outputs = []
        # input nodes
        for i in range(input_size):
            n = Genome.Node(("Sensor", "Input"), num=g.get_current_node_num()+1 )
            g.add_node(n)
            inputs.append(n)

        # output nodes
        for o in range(output_size):
            n = Genome.Node(("Input", "Output"), num=g.get_current_node_num()+1)
            g.add_node(n)
            outputs.append(n)
        #
        for i in inputs:
            for o in outputs:
                g.connect_nodes(i, o)
        return g



In [29]:

g = Genome.init_with_shape((2,1))

In [30]:
print(g)

Genome with 2 connections and 3 nodes
